In [ ]:
#import torch
#from transformers import pipeline



In [2]:
import pandas as pd

final_data = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/annotation_data_translated_with_blame.csv")


#df_trans.pop('Unnamed: 0')

final_data.head()



,Unnamed: 0,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,translated_text,blame_in_text
0,0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK,"The meeting is open. Under the Constitution, t...","[{'sequence': 'The meeting is open.', 'labels'..."
1,1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK,I would like to thank Things for the confidenc...,[{'sequence': 'I would like to thank Things fo...
2,2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK,"Twenty-five years ago, a majority of the peopl...","[{'sequence': 'Twenty-five years ago, a majori..."
3,3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK,From the Committee to the Committee of the Reg...,"[{'sequence': ""From the Committee to the Commi..."
4,4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK,We are faced with a very crucial parliamentary...,[{'sequence': 'We are faced with a very crucia...


In [3]:
test_data = final_data.loc[0:4]



In [47]:
single_list

'[{\'sequence\': \'I would like to thank Things for the confidence they have shown me by re-electing me as President.\', \'labels\': [\'praise\', \'neutral\', \'blame\'], \'scores\': [0.9563701152801514, 8.46099283080548e-07, 1.2345350342002348e-07]}, {\'sequence\': \'I shall make every effort, in close cooperation with my colleagues in the Bureau, to fulfil my duties as President of the Things.\', \'labels\': [\'neutral\', \'praise\', \'blame\'], \'scores\': [0.9998986124992371, 5.008435891795671e-06, 1.2553725525776827e-07]}, {\'sequence\': \'I welcome our guests today.\', \'labels\': [\'neutral\', \'blame\', \'praise\'], \'scores\': [0.9999908208847046, 2.363743078603875e-06, 1.7596008206055558e-07]}, {\'sequence\': \'I warmly welcome her Majesty the Queen, to His Royal Highness the Prince, to His Royal Highness Crown Prince Frederik, to His Royal Highness Prince Joachim, to Her Royal Highness Princess Alexandra and to Her Royal Highness Princess Benedikte.\', \'labels\': [\'praise\

In [ ]:
'''def extract_blame_from_paragraph(input):

        sentence_list = ast.literal_eval(input)

        blame_binary = []

        for sent_dict in sentence_list:
                label_scores = dict(zip(sent_dict['labels'], sent_dict['scores']))
                blame_probs = (label_scores.get('blame', 0.0))
                praise_probs = (label_scores.get('praise', 0.0))
                neutral_probs = (label_scores.get('neutral', 0.0))

                if (blame_probs >= praise_probs) & (blame_probs >= neutral_probs) & (blame_probs >= 0.80):
                        blame = 1
                else:
                        blame = 0
                
                blame_binary.append(blame)

        return blame_binary

final_data['blame_binary'] = final_data['blame_in_text'].apply(extract_blame_from_paragraph)'''

In [4]:
import ast
import swifter  # optional for parallel CPU usage

def extract_blame_from_paragraph_lookup(input_str):
    """
    Returns a binary list for blame per sentence:
    1 if blame is highest among labels and >= 0.8, else 0
    Handles arbitrary label order.
    """
    try:
        sentence_list = ast.literal_eval(input_str)
    except Exception:
        return []

    # List comprehension is faster than appending in a loop
    blame_binary = [
        int(
            (label_score := {label: score for label, score in zip(sent['labels'], sent['scores'])})['blame']
            >= max(label_score.get('praise', 0.0), label_score.get('neutral', 0.0), 0.8)
        )
        for sent in sentence_list
    ]

    return blame_binary


In [6]:
import ipywidgets

# Parallelize using swifter
final_data['blame_binary'] = final_data['blame_in_text'].swifter.apply(extract_blame_from_paragraph_lookup)




Pandas Apply:   0%|          | 0/36314 [00:00<?, ?it/s]

In [9]:
final_data.pop('Unnamed: 0')
final_data.to_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/annotation_data_translated_with_blame_heuristics.csv", index = False)

In [10]:
final_data.head()

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,translated_text,blame_in_text,blame_binary
0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK,"The meeting is open. Under the Constitution, t...","[{'sequence': 'The meeting is open.', 'labels'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK,I would like to thank Things for the confidenc...,[{'sequence': 'I would like to thank Things fo...,"[0, 0, 0, 0, 0, 0, 0, 0]"
2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK,"Twenty-five years ago, a majority of the peopl...","[{'sequence': 'Twenty-five years ago, a majori...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK,From the Committee to the Committee of the Reg...,"[{'sequence': ""From the Committee to the Commi...","[0, 0]"
4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK,We are faced with a very crucial parliamentary...,[{'sequence': 'We are faced with a very crucia...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
'''OBS: can quite easily (maybe) be made into a multiclassification model 
(blame/praise/neurtral). 
But due to timely constraints, the scope has been limited for this paper'''